In [45]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import sys
sys.path.append('../')
from qnetsur.simulatedannealing import simulated_annealing
from qnetsur.utils import Simulation

In [77]:
# minimize via simulated annealing. Note that it is a maximization algorithm - so the sign must be adjusted for minimization.
def minimize_sa(objfunc, n, bounds:list, btype:str, tlimit:float):
    vars = { # define variables and bounds for given simulation function 
        'range': {},
        'choice':{},
        'ordinal':{}
    }
    for i in range(n):
        vars['range'][f'x{i}'] = (bounds, btype)

    def simwrapper(sim, kwargs):
        x = list(kwargs.values())
        return [-sim(x)] # adjust sign
    sim = Simulation(simwrapper, sim=objfunc, values={}, variables=vars, rng=np.random.default_rng(seed=42))
    result = simulated_annealing(sim, beta_schedule=500, temp=5, limit=tlimit)
    result = pd.DataFrame.from_records(result)
    opt = result.iloc[result.objective.idxmax()]
    return opt

In [78]:
# continuous benchmark function 137 to be optimized (source: https://arxiv.org/pdf/1308.4008)
# cell takes about 33s to execute

def f137(x):
    return np.sum(np.square(x))

n = 3 # optimum at f(0,0,...)
bounds = [(-1, 1)] * n
x0 = np.random.uniform(-1,1,n)
res_bfgs = minimize(f137, x0, method='L-BFGS-B', bounds=bounds)
res_sa = minimize_sa(f137, n, [-1. , 1.], 'float', tlimit=0.01)

print('RESULTS FOR BFGS: \n', res_bfgs)
print('\nRESULTS FOR SA: \n', res_sa)

RESULTS FOR BFGS: 
   message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 4.700484288313155e-17
        x: [-6.631e-09  4.271e-11  1.743e-09]
      nit: 1
      jac: [-3.261e-09  1.009e-08  1.349e-08]
     nfev: 12
     njev: 3
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

RESULTS FOR SA: 
 x0           0.017513
x1          -0.005656
x2           0.007828
objective   -0.000400
time         0.000140
Name: 210152, dtype: float64


In [87]:
# discontinuous benchmark function 38 to be optimized (source: https://arxiv.org/pdf/1308.4008)
# cell takes about 51s to execute

def f38(x):
    term1 = -0.1 * np.sum(np.cos(5 * np.pi * np.array(x)))
    term2 = np.sum(np.square(x))
    return term1 + term2

n = 2 # optimum is -0.2, respectively at f(0,0)
bounds = [(-1, 1)] * n
x0 = np.random.uniform(-1,1,n)
res_bfgs = minimize(f38, x0, method='L-BFGS-B', bounds=bounds)
res_sa = minimize_sa(f38, n, [-1. , 1.], 'float', tlimit=0.02)

print('RESULTS FOR BFGS: \n', res_bfgs)
print('\nRESULTS FOR SA: \n', res_sa)

RESULTS FOR BFGS: 
   message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.38735659109503506
        x: [-7.251e-01  1.747e-07]
      nit: 9
      jac: [-8.105e-07  4.791e-06]
     nfev: 42
     njev: 14
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

RESULTS FOR SA: 
 x0          -0.000286
x1           0.000739
objective    0.199992
time         0.000124
Name: 26244, dtype: float64


In [88]:
# disc. benchmark function 139 to be optimized (source: https://arxiv.org/pdf/1308.4008)
# cell takes about 12s to execute

def f139(x):
    return np.sum((np.abs(x) - 0.5) ** 2)

n = 4 # optimum is 0, respectively at f(0.5,0.5,...)
bounds = [(-1, 1)] * n
x0 = np.random.uniform(-1,1,n)
res_bfgs = minimize(f139, x0, method='L-BFGS-B', bounds=bounds)
res_sa = minimize_sa(f139, n, [-1. , 1.], 'float', tlimit=0.005)

print('RESULTS FOR BFGS: \n', res_bfgs)
print('\nRESULTS FOR SA: \n', res_sa)

RESULTS FOR BFGS: 
   message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 1.2173608285176432e-16
        x: [-5.000e-01 -5.000e-01  5.000e-01 -5.000e-01]
      nit: 1
      jac: [ 2.422e-09  3.121e-09 -1.827e-09 -5.567e-09]
     nfev: 15
     njev: 3
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>

RESULTS FOR SA: 
 x0          -0.531012
x1          -0.517535
x2           0.503465
x3          -0.491062
objective   -0.001361
time         0.000188
Name: 74002, dtype: float64
